In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Data** **Preparation**

In [2]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 30.1 MB/s eta 0:00:00


In [3]:
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.9/113.9 kB 11.5 MB/s eta 0:00:00


In [4]:
import pandas as pd
import re
import nltk
import string
import emoji
import contractions
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [5]:
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
import pandas as pd

# Load the dataset
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/GAT/Mental-Health-Twitter.csv")

# Make sure the labels are balanced
positive_samples = df[df['label'] == 1].sample(n=1000, random_state=42)
negative_samples = df[df['label'] == 0].sample(n=1000, random_state=42)

# Combine and shuffle
reduced_df = pd.concat([positive_samples, negative_samples]).sample(frac=1, random_state=42)

# Save to new CSV
reduced_df.to_csv("reduced_dataset.csv", index=False)


In [7]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/GAT/reduced_dataset.csv')

In [8]:
print(df.head().to_string(index=False))


 Unnamed: 0            post_id                   post_created                                                                                                                                              post_text    user_id  followers  friends  favourites  statuses  retweets  label
      19115 813058119823552512 Sun Dec 25 16:25:36 +0000 2016                                                                                                             Lock Solid Joinery https://t.co/Cyqu0DJ6xI 1458225506       1197      701        1156     61856         0      0
       1794 592723571828465664 Mon Apr 27 16:14:46 +0000 2015                                                                                    Relaxation Techniques for Stress Relief |  | http://t.co/8BSQvxHfEZ 1616997456        475     1235           0     21533         0      1
      12029 810572263938473984 Sun Dec 18 19:47:42 +0000 2016                                                                                 Since the

In [9]:
#print("Columns:", df.columns)
if 'label' in df.columns and 'post_text' in df.columns:
    df = df.rename(columns={'post_text': 'text'})
    df = df[['text', 'label']]
print("Columns:", df.columns)

print(df.head().to_string(index=False))

df = df.sample(frac=1, random_state=42).reset_index(drop=True)

Columns: Index(['text', 'label'], dtype='object')
                                                                                                                                                  text  label
                                                                                                            Lock Solid Joinery https://t.co/Cyqu0DJ6xI      0
                                                                                   Relaxation Techniques for Stress Relief |  | http://t.co/8BSQvxHfEZ      1
                                                                                Since the #Lions offense can't score a TD, their defense will have to.      0
                                    RT @Ashton5SOS: HERE IT IS! My favorite video we've ever made!!! 😃✌🏼️http://t.co/Y2QUCGpHk9 http://t.co/iKbCnONBCF      1
RT @celtics: Al Horford is the league's only player avg. at least 15 PTS, 7 REB, 5 AST &amp; 2 BLK per game #NBAVOTE\r\n\r\nRT to vote! https://t.co/…      0


In [10]:
##Data cleaning(text)
def clean_text(text):
  text = re.sub(r'@\w+|#\w+|http\S+|www\S+', '', text) ## removing symols
  text = emoji.replace_emoji(text, replace='') ## removing emojis
  text = contractions.fix(text) ## expand contractions
  text = text.translate(str.maketrans('', '', string.punctuation)) ## femove punctuations
  text = text.lower()
  return text

In [11]:
## Data Preprocessing
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
#stop_words

def preprocess(text):
  tokens = word_tokenize(text) # tokernizer
  tokens = [word for word in tokens if word not in stop_words] # remove stop words
  tokens = [stemmer.stem(word) for word in tokens] # stemming
  return ' '.join(tokens)


In [12]:
df['cleaned_text'] = df['text'].apply(clean_text)
df['processed_text'] = df['cleaned_text'].apply(preprocess)
df = df[['processed_text', 'label']]
df.columns = ['text', 'label']
print(df.head().to_string())
df.to_csv('Preprocessed_Mental_Health_Tweets.csv', index=False) #saving preprocessed data(incase)
print(len(df))
print(df['label'].value_counts())

                                                                    text  label
0  rt clari sure need s1 clari build clari journey becom warrior meant …      0
1                   oh wow never realiz blue would make excel colour kan      0
2       publish place whydoctorsrecommendantidepressantsforsmallchildren      1
3                holidayinnexpress hotel us en jakarta jktwh hoteldetail      0
4      surgeon got realli surli last time put hold call code scari place      1
2000
label
0    1000
1    1000
Name: count, dtype: int64


**Embedding** **Extraction**

In [13]:
pip install transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [14]:
import torch
from transformers import BertTokenizer, BertModel
from transformers import RobertaTokenizer, RobertaModel
from transformers import DebertaTokenizer, DebertaModel
from tqdm import tqdm

In [15]:
# Loading pretrained models and tokenizer
bert_model = BertModel.from_pretrained('bert-base-uncased')
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

roberta_model = RobertaModel.from_pretrained('roberta-base')
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

deberta_model = DebertaModel.from_pretrained('microsoft/deberta-base')
deberta_tokenizer = DebertaTokenizer.from_pretrained('microsoft/deberta-base')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/559M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/559M [00:00<?, ?B/s]

In [16]:
# setting models to evaluation mode
bert_model.eval()
roberta_model.eval()
deberta_model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model.to(device)
roberta_model.to(device)
deberta_model.to(device)
device

device(type='cuda')

In [17]:
df = pd.read_csv("Preprocessed_Mental_Health_Tweets.csv")

In [18]:
MAX_LEN = 64

def get_embedding(text, tokenizer, model):
  if not isinstance(text, str):
    text = str(text)
  inputs = tokenizer(text, return_tensors='pt', max_length=MAX_LEN, truncation=True, padding='max_length') #preparing text for model
  inputs = {k: v.to(device) for k, v in inputs.items()}
  with torch.no_grad():
    outputs = model(**inputs)
    last_hidden = outputs.last_hidden_state #shape : (1, max_len, hidden_size)
    avg_pooled = last_hidden.mean(dim=1) #(1, hidden_size)
  return avg_pooled.squeeze().cpu() #removes batch dimension(hidden_size)


#extract embeddings for all tweets
bert_embeddings = []
roberta_embeddings = []
deberta_embeddings = []

print("Extracting embeddings...")

for text in tqdm(df['text']):
  bert_embed = get_embedding(text, bert_tokenizer, bert_model)
  roberta_embed = get_embedding(text, roberta_tokenizer, roberta_model)
  deberta_embed = get_embedding(text, deberta_tokenizer, deberta_model)

  bert_embeddings.append(bert_embed)
  roberta_embeddings.append(roberta_embed)
  deberta_embeddings.append(deberta_embed)

print("Embeddings extracted.")

Extracting embeddings...


100%|██████████| 2000/2000 [01:46<00:00, 18.82it/s]

Embeddings extracted.


In [19]:
# Convert to tensors
bert_tensor = torch.stack(bert_embeddings)
roberta_tensor = torch.stack(roberta_embeddings)
deberta_tensor = torch.stack(deberta_embeddings)

# Ensure same hidden size for BER, Roberta, Deberta(768)
assert bert_tensor.shape[1] == roberta_tensor.shape[1] == deberta_tensor.shape[1] == 768

#ensemble
stacked = torch.stack([bert_tensor, roberta_tensor, deberta_tensor], dim=2) #size : (num_samples, hidden_size, 3)

#flattening
fused_embeddings = stacked.mean(dim=2)

#save
torch.save(fused_embeddings, "Feature_Matrix.pt")
torch.save(torch.tensor(df['label'].values), "Labels.pt")

print("Embeddings extracted and saved.")

Embeddings extracted and saved.


**Graph Construction**

In [20]:
pip install networkx torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 58.5 MB/s eta 0:00:00


In [21]:
import networkx as nx
from torch_geometric.data import Data
import numpy as np
import torch
from tqdm import tqdm

In [23]:
# Load the feature matrix and labels
feature_matrix = torch.load("/content/Feature_Matrix.pt", weights_only=False)  # shape: (N, hidden_size)
labels = torch.load("/content/Labels.pt", weights_only=False)                  # shape: (N,)

print("Feature matrix:", feature_matrix.shape)
print("Labels:", labels.shape)

Feature matrix: torch.Size([2000, 768])
Labels: torch.Size([2000])


In [26]:
# constructing subgraphs
BATCH_SIZE = 500

all_subgraphs = []
num_samples = feature_matrix.shape[0]

print("constructing subgraphs...")

for start_idx in tqdm(range(0, num_samples, BATCH_SIZE)):
  end_idx = min(start_idx + BATCH_SIZE, num_samples)
  batch_feats = feature_matrix[start_idx:end_idx]  # (C, 768)

  adj_matrix = torch.mm(batch_feats, batch_feats.T)  # adjacency matrix(C, C)
  adj_np = adj_matrix.numpy()

  G = nx.Graph()
  G.add_nodes_from(range(start_idx, end_idx)) #build networkx subgraph

  for i in range(adj_np.shape[0]):
    for j in range(i+1, adj_np.shape[1]):
      weight = adj_np[i, j]
      if weight > 0:
        G.add_edge(start_idx + i, start_idx + j, weight = weight)

  all_subgraphs.append(G)

print("Subgraphs constructed.")

print("Merging subgraphs...")
full_graph = nx.compose_all(all_subgraphs) #merging subgraphs into a single graph
print("Subgraphs merged.")

constructing subgraphs...


100%|██████████| 4/4 [00:00<00:00,  4.95it/s]


Subgraphs constructed.
Merging subgraphs...
Subgraphs merged.


In [27]:
edge_index = torch.tensor(list(full_graph.edges), dtype=torch.long).t().contiguous() # converting to pytorch geometric format
x = feature_matrix
y = labels

data = Data(x=x, edge_index=edge_index, y=y)

# Save graph data
torch.save(data, "Graph_Data.pt")

print("✅ Graph construction complete.")
print("Graph stats:")
print(f"- Nodes: {data.num_nodes}")
print(f"- Edges: {data.num_edges}")
print(f"- Feature dim: {data.num_node_features}")


✅ Graph construction complete.
Graph stats:
- Nodes: 2000
- Edges: 499000
- Feature dim: 768


In [1]:
#!pip install torch-scatter
!pip install torch-sparse
!pip install scikit-learn
#pip install torch-geometric

  Using cached torch_sparse-0.6.18.tar.gz (209 kB)
  Preparing metadata (setup.py) ... done
ERROR: Operation cancelled by user
ERROR: Operation cancelled by user


KeyboardInterrupt: 

In [2]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from torch_geometric.data import Data
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import random
import numpy as np

In [4]:
# Load the graph data
data: Data = torch.load("/content/drive/MyDrive/Colab Notebooks/GAT/Graph_Data.pt", weights_only=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data = data.to(device)

In [5]:
def generate_masks(num_nodes, train_ratio=0.6, val_ratio=0.2, seed=42):
    np.random.seed(seed)
    indices = np.random.permutation(num_nodes)
    train_size = int(train_ratio * num_nodes)
    val_size = int(val_ratio * num_nodes)

    train_idx = indices[:train_size]
    val_idx = indices[train_size:train_size + val_size]
    test_idx = indices[train_size + val_size:]

    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    val_mask = torch.zeros(num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(num_nodes, dtype=torch.bool)

    train_mask[train_idx] = True
    val_mask[val_idx] = True
    test_mask[test_idx] = True

    return train_mask, val_mask, test_mask

In [6]:
data.train_mask, data.val_mask, data.test_mask = generate_masks(data.num_nodes)


In [7]:
#define GAT model
from torch.nn import Linear, Dropout

class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=1, dropout=0.5):
        super(GAT, self).__init__()
        self.gat1 = GATConv(in_channels, hidden_channels, heads=heads, dropout=dropout)
        self.gat2 = GATConv(hidden_channels * heads, out_channels, heads=1, concat=False, dropout=dropout)
        self.dropout = Dropout(dropout)

    def forward(self, x, edge_index):
        x = self.gat1(x, edge_index)
        x = F.elu(x)
        x = self.dropout(x)
        x = self.gat2(x, edge_index)
        return x

In [11]:
model = GAT(in_channels=data.num_node_features, hidden_channels=128, out_channels=2, heads=4).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
loss_fn = torch.nn.CrossEntropyLoss()


In [12]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = loss_fn(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

In [13]:
@torch.no_grad()
def evaluate():
    model.eval()
    logits = model(data.x, data.edge_index)
    preds = logits.argmax(dim=1)

    results = {}
    for split in ['train', 'val', 'test']:
        mask = data[f'{split}_mask']
        y_true = data.y[mask].cpu()
        y_pred = preds[mask].cpu()
        results[split] = {
            'acc': accuracy_score(y_true, y_pred),
            'f1': f1_score(y_true, y_pred),
            'precision': precision_score(y_true, y_pred),
            'recall': recall_score(y_true, y_pred)
        }
    return results

In [19]:
for epoch in range(1, 1001):  # 50 epochs
    loss = train()
    if epoch % 50 == 0:
        metrics = evaluate()
        print(f"[Epoch {epoch:02d}] Loss: {loss:.4f} | "
              f"Val Acc: {metrics['val']['acc']:.4f} | "
              f"F1: {metrics['val']['f1']:.4f}")

[Epoch 50] Loss: 0.6882 | Val Acc: 0.5000 | F1: 0.5122
[Epoch 100] Loss: 0.6850 | Val Acc: 0.5200 | F1: 0.3885
[Epoch 150] Loss: 0.6857 | Val Acc: 0.5075 | F1: 0.3088
[Epoch 200] Loss: 0.6856 | Val Acc: 0.5075 | F1: 0.5817
[Epoch 250] Loss: 0.6859 | Val Acc: 0.5025 | F1: 0.5683
[Epoch 300] Loss: 0.6868 | Val Acc: 0.5150 | F1: 0.5198
[Epoch 350] Loss: 0.6855 | Val Acc: 0.5025 | F1: 0.4777
[Epoch 400] Loss: 0.6838 | Val Acc: 0.4950 | F1: 0.3988
[Epoch 450] Loss: 0.6834 | Val Acc: 0.4950 | F1: 0.3841
[Epoch 500] Loss: 0.6884 | Val Acc: 0.5150 | F1: 0.5907
[Epoch 550] Loss: 0.6856 | Val Acc: 0.5000 | F1: 0.5327
[Epoch 600] Loss: 0.6820 | Val Acc: 0.5200 | F1: 0.5026
[Epoch 650] Loss: 0.6887 | Val Acc: 0.5050 | F1: 0.6551
[Epoch 700] Loss: 0.6850 | Val Acc: 0.5000 | F1: 0.2908
[Epoch 750] Loss: 0.6858 | Val Acc: 0.5000 | F1: 0.6324
[Epoch 800] Loss: 0.6856 | Val Acc: 0.5050 | F1: 0.6148
[Epoch 850] Loss: 0.6833 | Val Acc: 0.5175 | F1: 0.6352
[Epoch 900] Loss: 0.6850 | Val Acc: 0.5000 | F1: 

In [20]:
# Step 5: Final test evaluation
final_metrics = evaluate()
print("\n✅ Final Test Metrics:")
for metric, value in final_metrics['test'].items():
    print(f"{metric.capitalize()}: {value:.4f}")


✅ Final Test Metrics:
Acc: 0.4825
F1: 0.6336
Precision: 0.4799
Recall: 0.9323
